In [127]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [128]:
import pandas as pd

# Read the dataset
dataset = pd.read_json("data_without_hashtags.json")

# Assuming "is_sarcastic" is the column you're interested in
sarcastic_counts = dataset['isSarcastic'].value_counts()

# Display the result
print("Number of rows for each value in the 'isSarcastic' column:")
print(sarcastic_counts)

Number of rows for each value in the 'isSarcastic' column:
isSarcastic
0    21292
1    18488
Name: count, dtype: int64


In [148]:
dataset.head()

,text,isSarcastic
0,@0430yes i hope youre lurking rn. i want to li...,0
1,05 really taught me a valuable lesson I'm neve...,0
2,"@098BERRY Never had a voice to protest, so you...",0
3,@0hMySt4rs Rest in peace & love to you and you...,0
4,100 days until Christmas! 🌲 #too soon ready yet,0


In [4]:
###################################################################################

#Preprocessing starts here


#stop words removed

In [129]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sarah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sarah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [130]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create DataFrame
df = pd.DataFrame(dataset)

# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply remove_stopwords function to the 'text' column
df['text_without_stopwords'] = df['text'].apply(remove_stopwords)

print("Tokenized texts:")
print(df['text'])


Tokenized texts:
0        @0430yes i hope youre lurking rn. i want to li...
1        05 really taught me a valuable lesson I'm neve...
2        @098BERRY Never had a voice to protest, so you...
3        @0hMySt4rs Rest in peace & love to you and you...
4         100 days until Christmas! 🌲 #too soon  ready yet
                               ...                        
39775    @Zendaya I could see the makeup artists giving...
39776    @ZiggiWatkins11 Slvr... That's great name #NOT...
39777    @zoso4986 @Nero He is the fag we need but not ...
39778    Zuma sounding like Kanye West right now trying...
39779    @ZZUCRU @UWDawgPack So true. Students - stick ...
Name: text, Length: 39780, dtype: object


In [131]:
dataset = df

In [ ]:
###############################################################################replace emojis and emoticons

In [23]:
%pip install emoji

   ---------------------------------------- 0.0/433.8 kB ? eta -:--:--
    --------------------------------------- 10.2/433.8 kB ? eta -:--:--
    --------------------------------------- 10.2/433.8 kB ? eta -:--:--
   ------ -------------------------------- 71.7/433.8 kB 653.6 kB/s eta 0:00:01
   ------------- -------------------------- 143.4/433.8 kB 1.1 MB/s eta 0:00:01
   ------------- -------------------------- 143.4/433.8 kB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 276.5/433.8 kB 1.1 MB/s eta 0:00:01
   --------------------------------- ------ 368.6/433.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 433.8/433.8 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [132]:
import pandas as pd
import emoji
import re
import json

#df = pd.read_json("data_without_hashtags.json")
df = dataset

# Function to replace emojis with words
def replace_emojis(text):
    return emoji.demojize(text, delimiters=(" ", " "))  # Ensure emojis are separated by spaces


def replace_emoticons(text):
    emoticon_dict = {
    ':)': 'smile',
    ':(': 'sad',
    ':D': 'big smile',
    ':P': 'tongue out',
    ';)': 'wink',
    ':O': 'surprise',
    ':|': 'neutral',
    ':/': 'uncertain',
    ":'(": 'tears of sadness',
    ":'D": 'tears of joy',
    ':*': 'kiss',
    ':@': 'angry',
    ':x': 'mouth shut',
    ':3': 'cute',
    ':$': 'embarrassed',
    ":')": 'single tear',
    ':p': 'tongue out'
}


    # #Construct regex pattern using re.escape() to escape special characters
    # pattern = re.compile(r'(' + '|'.join(re.escape(emoticon) for emoticon in emoticon_dict.keys()) + ')', re.IGNORECASE)

    # # Replace emoticons using the pattern
    # return pattern.sub(lambda match: emoticon_dict.get(match.group().lower(), match.group()), text)

    # Convert emoticon keys to lowercase
    emoticon_dict_lower = {key.lower(): value for key, value in emoticon_dict.items()}

    # Construct regex pattern using re.escape() to escape special characters
    pattern = re.compile(r'(' + '|'.join(re.escape(emoticon) for emoticon in emoticon_dict_lower.keys()) + ')', re.IGNORECASE)

    # Replace emoticons using the pattern
    return pattern.sub(lambda match: emoticon_dict_lower.get(match.group().lower(), match.group()), text)




# Apply functions to replace emojis and emoticons and update DataFrame columns
df['text'] = df['text'].apply(replace_emojis)
df['text'] = df['text'].apply(replace_emoticons)

# Display the DataFrame
print(df)

data_dict = df.to_dict()

with open('removedEmoji.json', 'w') as f:
    json.dump(data_dict, f, indent=4)


                                                    text  isSarcastic  \
0      @0430yes i hope youre lurking rn. i want to li...            0   
1      05 really taught me a valuable lesson I'm neve...            0   
2      @098BERRY Never had a voice to protest, so you...            0   
3      @0hMySt4rs Rest in peace & love to you and you...            0   
4      100 days until Christmas!  evergreen_tree  #to...            0   
...                                                  ...          ...   
39775  @Zendaya I could see the makeup artists giving...            1   
39776  @ZiggiWatkins11 Slvr... That's great name #NOT...            1   
39777  @zoso4986 @Nero He is the fag we need but not ...            1   
39778  Zuma sounding like Kanye West right now trying...            1   
39779  @ZZUCRU @UWDawgPack So true. Students - stick ...            1   

                                  text_without_stopwords  
0      @ 0430yes hope youre lurking rn . want listen ...  
1    

In [133]:
dataset = df

In [ ]:
################################################################################# remove @s of users

removes @s

In [106]:
import pandas as pd
import re

def remove_user_mentions(text):
    # Define the regex pattern to match @ mentions followed by numbers
    pattern = re.compile(r'@\d+')

    # Remove @ mentions using the pattern
    return pattern.sub('', text)

# Load the DataFrame
#df = pd.read_json("data_without_hashtags.json")

df = dataset

# Apply remove_user_mentions function to the 'text' column
df['text'] = df['text'].apply(remove_user_mentions)

# Save the updated DataFrame if needed
df.to_json("updated_data_without_mentions.json", orient='records', lines=True)

dataset = df


removes @ and then replaces it with word person

In [134]:
import pandas as pd
import re

def remove_user_mentions(text):
    # Define the regex pattern to match @ mentions followed by numbers
    pattern = re.compile(r'@\d+')

    # Remove @ mentions using the pattern
    return pattern.sub('person', text)

# Load the DataFrame
#df = pd.read_json("data_without_hashtags.json")

df = dataset

# Apply remove_user_mentions function to the 'text' column
df['text'] = df['text'].apply(remove_user_mentions)

# Save the updated DataFrame if needed
df.to_json("updated_data_without_mentions.json", orient='records', lines=True)

dataset = df

In [ ]:
############################################################################
#spellcheck words

In [10]:
%pip install pyspellchecker



   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB 653.6 kB/s eta 0:00:11
   - -------------------------------------- 0.2/6.8 MB 1.7 MB/s eta 0:00:04
   - -------------------------------------- 0.3/6.8 MB 2.1 MB/s eta 0:00:04
   --- ------------------------------------ 0.5/6.8 MB 3.0 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/6.8 MB 2.8 MB/s eta 0:00:03
   ----- ---------------------------------- 0.9/6.8 MB 3.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.2/6.8 MB 3.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.4/6.8 MB 3.7 MB/s eta 0:00:02
   --------- ------------------------------ 1.6/6.8 MB 3.9 MB/s eta 0:00:02
   ---------- ----------------------------- 1.7/6.8 MB 3.7 MB/s eta 0:00:02
   ----------- ---------------------------- 2.0/6.8 MB 3.8 MB/s eta 0:00:02
   ------------ -------

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset
dataset = pd.read_json("data_without_hashtags.json")

# Split the indices of the DataFrame into two halves with stratified sampling
indices_half1, indices_half2 = train_test_split(dataset.index, test_size=0.5, stratify=dataset['isSarcastic'], random_state=42)

# Create DataFrame quarters using the selected indices
dataset_half = dataset.loc[indices_half1]

In [40]:
import pandas as pd

# Read the dataset
dataset = pd.read_json("data_without_hashtags.json")

# Sample 150 rows from the dataset
sampled_dataset = dataset.sample(n=150, random_state=42)

# Display the sampled dataset
print(sampled_dataset)


                                                    text  isSarcastic
13906  Now serving up great coffee from @topecacoffee...            0
27947  I'm assuming Greek yogurt is just regular yogu...            1
16807  Spread love In this Evening with Manwa Laage.....            0
34788  RT The only benefit of dating a Srilankan girl...            1
22679  @BudGirl555 awesome. I cannot wait. Overdue. P...            1
...                                                  ...          ...
36254  Thank God I still have 5 days to achieve my go...            1
38668  when you know you typed ur password wrong but ...            1
22865  Can't wait to celebrate my birthday in boot ca...            1
17210  @tedcruz say a prayer cause no matter who is e...            0
734            And I thought I was really sarcastic but.            0

[150 rows x 2 columns]


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset
dataset = pd.read_json("data_without_hashtags.json")

indices_half1, indices_half2 = train_test_split(dataset.index, test_size=0.5, stratify=dataset['isSarcastic'], random_state=42)

# Split each half into two quarters
indices_quarter1, indices_quarter2 = train_test_split(indices_half1, test_size=0.5, stratify=dataset.loc[indices_half1]['isSarcastic'], random_state=42)
indices_quarter3, indices_quarter4 = train_test_split(indices_half2, test_size=0.5, stratify=dataset.loc[indices_half2]['isSarcastic'], random_state=42)

# Create DataFrame quarters using the selected indices
dataset_quarter1 = dataset.loc[indices_quarter1]

In [41]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sampled_dataset['text'], sampled_dataset['isSarcastic'], test_size=0.2, random_state=42)

dataset = sampled_dataset

In [34]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset_half['text'], dataset_half['isSarcastic'], test_size=0.2, random_state=42)

dataset = dataset_half

In [42]:
from spellchecker import SpellChecker
import pandas as pd

test_text = "This is a test #hdsjs"

spell = SpellChecker()
def correct_spelling(text):
    corrected_text = []
    words = text.split()
    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word:
            corrected_text.append(corrected_word)
        else:
            corrected_text.append(word)  # Keep the original word if no correction is found
    return ' '.join(corrected_text)


dataset['text'] = dataset['text'].apply(correct_spelling)

#corrected_text = correct_spelling(test_text)
#print(corrected_text)

# Convert DataFrame to JSON and write to a file
dataset.to_json('processed_data.json', orient='records', lines=True)


In [ ]:
#############################################################################
#abreviations

In [7]:
###########################################################################################

In [135]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Read the dataset
#dataset = pd.read_json("data_without_hashtags.json")

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['isSarcastic'], test_size=0.2, random_state=42)

# Tokenize and vectorize the training text data using Tokenizer and pad_sequences
max_length = 140
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length)

y_train_categorical = to_categorical(y_train, num_classes=2)

# Display the shapes of the resulting matrices
print("Shape of X_train:", X_train.shape)

# Tokenize and vectorize the testing text data using the same Tokenizer
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length)

# Display the shape of X_test
print("Shape of X_test:", X_test.shape)

Shape of X_train: (31824, 140)
Shape of X_test: (7956, 140)


In [136]:
from keras.models import Sequential
from keras.layers import Embedding, CuDNNLSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from keras.optimizers import Adam

embedding_dim = 100

# Define the vocabulary size based on the actual number of unique words in the training data
vocab_size = len(tokenizer.word_index) + 1

max_length = 140

optimizer = Adam(learning_rate=0.000009)
m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(CuDNNLSTM(units=150))
m1.add(Dense(units=64))
m1.add(Dense(units=64))
m1.add(Dense(units=1, activation='sigmoid'))

m1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
m1.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 140, 100)          3159400   
                                                                 
 cu_dnnlstm_17 (CuDNNLSTM)   (None, 150)               151200    
                                                                 
 dense_48 (Dense)            (None, 64)                9664      
                                                                 
 dense_49 (Dense)            (None, 64)                4160      
                                                                 
 dense_50 (Dense)            (None, 1)                 65        
                                                                 
Total params: 3,324,489
Trainable params: 3,324,489
Non-trainable params: 0
_________________________________________________________________


In [137]:
# Train the model
m1.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = m1.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20
498/498 [==============================] - 10s 18ms/step - loss: 0.6910 - accuracy: 0.5296 - val_loss: 0.6889 - val_accuracy: 0.5368
Epoch 2/20
498/498 [==============================] - 9s 17ms/step - loss: 0.6882 - accuracy: 0.5349 - val_loss: 0.6869 - val_accuracy: 0.5368
Epoch 3/20
498/498 [==============================] - 9s 18ms/step - loss: 0.6853 - accuracy: 0.5377 - val_loss: 0.6834 - val_accuracy: 0.5456
Epoch 4/20
498/498 [==============================] - 9s 18ms/step - loss: 0.6794 - accuracy: 0.5782 - val_loss: 0.6765 - val_accuracy: 0.5900
Epoch 5/20
498/498 [==============================] - 9s 18ms/step - loss: 0.6659 - accuracy: 0.6214 - val_loss: 0.6599 - val_accuracy: 0.6276
Epoch 6/20
498/498 [==============================] - 9s 17ms/step - loss: 0.6224 - accuracy: 0.6764 - val_loss: 0.6076 - val_accuracy: 0.6781
Epoch 7/20
498/498 [==============================] - 9s 17ms/step - loss: 0.5528 - accuracy: 0.7286 - val_loss: 0.5700 - val_accuracy: 0.701

In [138]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
y_val_pred_prob_m1 = m1.predict(X_test)
y_val_pred_m1 = (y_val_pred_prob_m1 > 0.5).astype(int)  # Threshold for binary classification

# Assuming y_test is in binary format (0 or 1)
y_val_true_m1 = y_test

# Calculate precision and recall for binary classification
precision_m1 = precision_score(y_val_true_m1, y_val_pred_m1)
recall_m1 = recall_score(y_val_true_m1, y_val_pred_m1)

# print the results
print(f'Precision: {precision_m1:.4f}')
print(f'Recall: {recall_m1:.4f}')

249/249 [==============================] - 1s 4ms/step
Precision: 0.7418
Recall: 0.7444
